# selection_queue

> Selection queue component with drag-drop reordering

In [ ]:
#| default_exp components.selection_queue

In [ ]:
#| export
from typing import Any, Dict, List
import json

from fasthtml.common import Div, Span, Button, Ul, Li, P, Hidden

# DaisyUI components
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, min_h
from cjm_fasthtml_tailwind.utilities.typography import (
    font_size, font_weight, font_family, text_align, truncate, list_style
)
from cjm_fasthtml_tailwind.utilities.layout import overflow
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.utilities.interactivity import cursor
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, grow, shrink
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Lucide icons
from cjm_fasthtml_lucide_icons.factory import lucide_icon

# Local imports
from cjm_transcript_source_select.html_ids import SelectionHtmlIds

## Queue Item

In [ ]:
#| export
def _render_queue_item(
    source: Dict[str, str],  # Source dict with record_id and provider_id
    index: int,  # Position in queue (1-based)
    remove_url: str,  # URL for removing from queue
) -> Any:  # Queue item element
    """Render a single item in the selection queue."""
    record_id = source.get("record_id", "")
    provider_id = source.get("provider_id", "")
    
    return Li(
        # Hidden input for form submission during reorder
        Hidden(name="item", value=record_id),
        
        # Drag handle
        Span(
            lucide_icon("grip-vertical", size=4, cls=str(text_dui.base_content.opacity(40))),
            cls=combine_classes("drag-handle", cursor.move, p.r(2))
        ),
        
        # Position number
        Span(f"{index}.", cls=combine_classes(font_weight.bold, p.r(2), w(6))),
        
        # Job ID
        Span(
            record_id[:10] + "..." if len(record_id) > 10 else record_id,
            cls=combine_classes(grow(), font_size.sm, font_family.mono, truncate),
            title=record_id
        ),
        
        # Remove button
        Button(
            lucide_icon("x", size=4, cls=str(text_dui.base_content.opacity(60))),
            cls=combine_classes(btn, btn_styles.ghost, btn_sizes.xs, m.l(1)),
            hx_post=remove_url,
            hx_vals=json.dumps({"record_id": record_id}),
            hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            hx_swap="outerHTML",
            data_action="remove",
            title="Remove from queue"
        ),
        
        id=SelectionHtmlIds.queue_item(record_id, provider_id),
        cls=combine_classes(
            "queue-item",
            flex_display,
            items.center,
            m.x(0),  # No horizontal margin
            m.y(1),  # Vertical margin for focus ring visibility
            p(3),    # Uniform padding
            bg_dui.base_100,
            bg_dui.base_200.hover
        ),
        data_record_id=record_id,
        data_provider_id=provider_id
    )

## Selection Queue

In [ ]:
#| export
def _render_selection_queue(
    selected_sources: List[Dict[str, str]],  # List of selected sources in order
    remove_url: str,  # URL for removing from queue
    reorder_url: str,  # URL for reordering queue
    clear_url: str,  # URL for clearing all
) -> Any:  # Queue panel component
    """Render the selection queue panel with drag-drop reordering."""
    queue_count = len(selected_sources)
    
    # Queue items
    queue_items = [
        _render_queue_item(source, i + 1, remove_url)
        for i, source in enumerate(selected_sources)
    ]
    
    # Empty state
    empty_state = Div(
        P(
            "No sources selected",
            cls=combine_classes(text_dui.base_content.opacity(40), text_align.center, font_size.sm)
        ),
        P(
            "Select transcriptions from the browser",
            cls=combine_classes(text_dui.base_content.opacity(30), text_align.center, font_size.xs)
        ),
        id=SelectionHtmlIds.QUEUE_EMPTY,
        cls=combine_classes(p(8), flex_display, flex_direction.col, justify.center, items.center, grow())
    ) if not selected_sources else None
    
    return Div(
        # Header
        Div(
            Div(
                Span("Selected", cls=str(font_weight.bold)),
                Span(
                    str(queue_count),
                    cls=combine_classes(badge, badge_colors.primary, badge_sizes.sm, m.l(2))
                ),
                cls=str(flex_display)
            ),
            Button(
                "Clear",
                cls=combine_classes(btn, btn_styles.ghost, btn_sizes.xs),
                hx_post=clear_url,
                hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
                hx_swap="outerHTML"
            ) if selected_sources else None,
            cls=combine_classes(
                flex_display, justify.between, items.center,
                p(3), border_dui.base_200, border.b()
            )
        ),
        
        # Queue list with native Sortable.js (initialized via script)
        Ul(
            *queue_items,
            id=SelectionHtmlIds.QUEUE_LIST,
            hx_post=reorder_url,
            hx_trigger="end",
            hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.QUEUE_CONTAINER),
            hx_swap="outerHTML",
            cls=combine_classes("sortable", grow(), overflow.y.auto, list_style.none, m(0), p(0))
        ) if selected_sources else empty_state,
        
        id=SelectionHtmlIds.QUEUE_CONTAINER,
        cls=combine_classes(
            # Responsive width: full on mobile, fixed on large screens
            w.full,
            w(64).lg,
            shrink(0).lg,
            min_h(48),
            # Base styling
            bg_dui.base_100,
            border_radius.box,
            shadow.lg,
            border_dui.base_300,
            flex_display,
            flex_direction.col,
            overflow.hidden
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()